In [1]:
import cv2
import mediapipe as mp
import math
import time
from rembg import remove
from PIL import Image, ImageChops
import os

cap = None
calculated_values = {}
height = 0

def pose_estimation():
    global cap
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose()
    mp_drawing = mp.solutions.drawing_utils
    start_time = time.time()

    while True:
        if cap is not None:
            ret, frame = cap.read()
            if not ret:
                break
            frame = cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)
            # Convert the frame to RGB format
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            # Perform pose estimation on the frame
            results = pose.process(frame_rgb)

            # Annotate the frame with pose estimation
            frame = cv2.cvtColor(frame_rgb, cv2.COLOR_RGB2BGR)
            # print(results.pose_world_landmarks)
            # print(results.pose_world_landmarks.RI
            if results.pose_landmarks:
                landmarks = results.pose_landmarks.landmark
                mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                          mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                          mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                          )
                def dist_bw_two_points(a, b, frame):
                    a_marks = landmarks[a]
                    b_marks = landmarks[b]

                    a_x, a_y = int(a_marks.x * frame.shape[1]), int(a_marks.y * frame.shape[0])
                    b_x, b_y = int(b_marks.x * frame.shape[1]), int(b_marks.y * frame.shape[0])

                    distance = math.sqrt((b_x - a_x) ** 2 + (b_y - a_y) ** 2)
                    cv2.circle(frame, (a_x, a_y), 5, (0, 255, 0), -1)
                    cv2.circle(frame, (b_x, b_y), 5, (0, 255, 0), -1)
                    
                    return distance

                # Relative Positioning of LEFT_WRIST AND RIGHT_WRIST
                fixed_distance = 80 / dist_bw_two_points(mp_pose.PoseLandmark.RIGHT_WRIST, mp_pose.PoseLandmark.LEFT_WRIST, frame)
                #print(fixed_distance)
                # LEFT ARM LENGTH
                left_arm_length = fixed_distance * dist_bw_two_points(mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.LEFT_WRIST, frame)
                
                # RIGHT ARM LENGTH
                right_arm_length = fixed_distance * dist_bw_two_points(mp_pose.PoseLandmark.RIGHT_SHOULDER, mp_pose.PoseLandmark.RIGHT_WRIST, frame)

                # LEFT LEG LENGTH
                #left_thigh_length = fixed_distance * dist_bw_two_points(mp_pose.PoseLandmark.LEFT_HIP, mp_pose.PoseLandmark.LEFT_KNEE, frame)
                #left_knee_length = fixed_distance * dist_bw_two_points(mp_pose.PoseLandmark.LEFT_FOOT_INDEX, mp_pose.PoseLandmark.LEFT_KNEE, frame)
                
                # RIGHT LEG LENGTH
                #right_thigh_length = fixed_distance * dist_bw_two_points(mp_pose.PoseLandmark.RIGHT_HIP, mp_pose.PoseLandmark.RIGHT_KNEE, frame)
                #right_knee_length = fixed_distance * dist_bw_two_points(mp_pose.PoseLandmark.RIGHT_FOOT_INDEX, mp_pose.PoseLandmark.RIGHT_KNEE, frame)
                
                # SHOULDER LENGTH
                shoulder_length = fixed_distance * dist_bw_two_points(mp_pose.PoseLandmark.RIGHT_SHOULDER, mp_pose.PoseLandmark.LEFT_SHOULDER, frame) + 3
                chest_circumference = 3.14 * shoulder_length
                
                # WAIST CIRCUMFERENCE
                waist_length = fixed_distance * dist_bw_two_points(mp_pose.PoseLandmark.RIGHT_HIP, mp_pose.PoseLandmark.LEFT_HIP, frame) + 4.86
                waist_circumference = 3.14 * waist_length
                
                # HEIGHT
                height = fixed_distance * dist_bw_two_points(mp_pose.PoseLandmark.NOSE, mp_pose.PoseLandmark.LEFT_HEEL,
                                                            frame) + 2 * fixed_distance * dist_bw_two_points(mp_pose.PoseLandmark.NOSE,
                                                                                                             mp_pose.PoseLandmark.LEFT_EYE_INNER, frame)
                
                height2 = (fixed_distance * dist_bw_two_points(mp_pose.PoseLandmark.NOSE, mp_pose.PoseLandmark.LEFT_HEEL, frame) +
                  fixed_distance * dist_bw_two_points(mp_pose.PoseLandmark.NOSE, mp_pose.PoseLandmark.LEFT_HIP, frame))
        
                # SITTING HEIGHT
                #sitting_height = height - (left_thigh_length + right_thigh_length) / 2
                
                # FOOT LENGTH
                #foot_length = fixed_distance * dist_bw_two_points(mp_pose.PoseLandmark.LEFT_FOOT_INDEX, mp_pose.PoseLandmark.LEFT_HEEL, frame)

                calculated_values['fixed_distance'] = str(round(2, 2)) + " cm"
                calculated_values['height'] = str(round(height, 2)) + " cm"
                calculated_values['height2'] = str(round(height2, 2)) + " cm"
                calculated_values['left_arm_length'] = str(round(left_arm_length, 2)) + " cm"
                #calculated_values['left_thigh_length'] = str(round(left_thigh_length, 2)) + " cm"
                #calculated_values['left_knee_length'] = str(round(left_knee_length, 2)) + " cm"
                calculated_values['right_arm_length'] = str(round(right_arm_length, 2)) + " cm"
                #calculated_values['right_thigh_length'] = str(round(right_thigh_length, 2)) + " cm"
                #calculated_values['right_knee_length'] = str(round(right_knee_length, 2)) + " cm"
                calculated_values['shoulder_length'] = str(round(shoulder_length, 2)) + " cm"
                calculated_values['chest_circumference'] = str(round(chest_circumference, 2)) + " cm"
                calculated_values['waist_length'] = str(round(waist_length, 2)) + " cm"
                calculated_values['waist_circumference'] = str(round(waist_circumference, 2)) + " cm"


                #calculated_values['sitting_height'] = str(round(sitting_height, 2)) + " cm"
                #calculated_values['foot_length'] = str(round(foot_length, 2)) + " cm"

            # Display the annotated frame
            cv2.imshow('Pose Estimation', frame)

            # Break the loop if 'q' is pressed or 15 seconds have passed
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
            if time.time() - start_time > 15:
                # Save the current frame as an image
                cv2.imwrite('captured_frame.jpg', frame)
                break

    cap.release()
    cv2.destroyAllWindows()
    #print(calculated_values)

    return results

def start_video():
    global cap
    if cap is None or not cap.isOpened():
        cap = cv2.VideoCapture(1)
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, 3840)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 2160)

def stop_video():
    global cap
    if cap is not None:
        cap.release()
        cap = None
        #print(calculated_values)

def remove_background(input_path):
    # Extract the directory and filename from the input path
    directory = os.path.dirname(input_path)
    filename = os.path.basename(input_path)

    # Create the output filename
    output_filename = os.path.splitext(filename)[0] + '_output.png'
    output_path = os.path.join(directory, output_filename)

    # Process the image
    input_image = Image.open(input_path)

    # Remove the background from the given image
    output_image = remove(input_image)
 
    # Save the image in the given path
    output_image.save(output_path)

    print(f"Processed image saved as {output_path}")


In [2]:
results = []

start_video()
results = pose_estimation()
stop_video()

# Remove background from the captured frame
# remove_background('captured_frame.jpg')



INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1716584014.597341   16712 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1716584014.614867   16712 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
/home/jetson/anaconda3/lib/python3.11/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
QObject::moveToThread: Current thread (0x16b1e990) is not the object's thread (0x19ad7ad0).
Cannot move to target thread (0x16b1e990)

QObject::moveToThread: Current thread (0x16b1e990) is not the object's thread (0x19ad7ad0).
Cannot move to target threa

In [3]:
print(results.pose_world_landmarks.landmark[15])
print(results.pose_world_landmarks.landmark[16])

dx = results.pose_world_landmarks.landmark[15].x - results.pose_world_landmarks.landmark[16].x
dy = results.pose_world_landmarks.landmark[15].y - results.pose_world_landmarks.landmark[16].y
dz = results.pose_world_landmarks.landmark[15].z - results.pose_world_landmarks.landmark[16].z

distance = dx * dx + dy * dy + dz * dz

print(f"distance {distance}")


x: 0.2764540910720825
y: -0.9170682430267334
z: -0.3814021944999695
visibility: 0.9865368008613586

x: -0.3099749982357025
y: -0.9260579943656921
z: -0.2678791284561157
visibility: 0.9628998041152954

distance 0.3568673789394916
